In [20]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [21]:
class SigmoidNeuron:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        self.weights = np.zeros(X.shape[1])
        self.bias = 0
        
        for _ in range(self.n_iterations):
            z = np.dot(X, self.weights) + self.bias
            predictions = self.sigmoid(z)
            
            dz = predictions - y
            dw = (1/len(X)) * np.dot(X.T, dz)
            db = (1/len(X)) * np.sum(dz)
            
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
    
    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        predictions = self.sigmoid(z)
        return (predictions >= 0.5).astype(int)

In [22]:
wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
classifiers = []
n_classes = len(np.unique(y))

In [24]:
for class_label in range(n_classes):
    y_train_binary = (y_train == class_label).astype(int)
    
    clf = SigmoidNeuron(learning_rate=0.01, n_iterations=1000)
    clf.fit(X_train_scaled, y_train_binary)
    classifiers.append(clf)

def predict_class(X, classifiers):
    predictions = np.array([clf.predict(X) for clf in classifiers])
    return np.argmax(predictions, axis=0)

y_train_pred = predict_class(X_train_scaled, classifiers)
y_test_pred = predict_class(X_test_scaled, classifiers)

In [25]:
def calculate_metrics(y_true, y_pred, class_label):
    y_true_binary = (y_true == class_label).astype(int)
    y_pred_binary = (y_pred == class_label).astype(int)
    
    accuracy = accuracy_score(y_true_binary, y_pred_binary)
    precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)
    
    return accuracy, precision, recall

print("Training Set Metrics:")
print("Class\tAccuracy\tPrecision\tRecall")
print("-" * 50)
for i in range(n_classes):
    acc, prec, rec = calculate_metrics(y_train, y_train_pred, i)
    print(f"{i}\t{acc:.3f}\t\t{prec:.3f}\t\t{rec:.3f}")

print("\nTest Set Metrics:")
print("Class\tAccuracy\tPrecision\tRecall")
print("-" * 50)
for i in range(n_classes):
    acc, prec, rec = calculate_metrics(y_test, y_test_pred, i)
    print(f"{i}\t{acc:.3f}\t\t{prec:.3f}\t\t{rec:.3f}")

Training Set Metrics:
Class	Accuracy	Precision	Recall
--------------------------------------------------
0	0.979		0.938		1.000
1	0.972		1.000		0.930
2	0.993		0.976		1.000

Test Set Metrics:
Class	Accuracy	Precision	Recall
--------------------------------------------------
0	1.000		1.000		1.000
1	1.000		1.000		1.000
2	1.000		1.000		1.000
